In [1]:
import json

from flow.core.params import (
    NetParams,
    InitialConfig,
    SumoParams,
    EnvParams,
    VehicleParams,
)
from flow.controllers import IDMController, ContinuousRouter, RLController
from flow.networks.figure_eight import FigureEightNetwork, ADDITIONAL_NET_PARAMS
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray.rllib.algorithms.ppo import DEFAULT_CONFIG
from ray.tune import run_experiments
from ray.tune.registry import register_env


In [2]:
# Define horizon as a variable to ensure consistent use across notebook
HORIZON = 100
N_WORKERS = 9  # 1 - number of CPUS
N_ROLLOUTS = 1
# The algorithm or model to train. This may refer to the name of a built-on
# algorithm (e.g. RLLib's DQN or PPO), or a user-defined trainable function or
# class registered in the tune registry.)
RL_ALG_NAME = "PPO"
EXP_NAME = "figure_eight"


In [3]:
vehicles = VehicleParams()
vehicles.add(
    "human",
    acceleration_controller=(IDMController, {}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=15,
)
vehicles.add(
    veh_id="rl",
    acceleration_controller=(RLController, {}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=1,
)


In [4]:
# Creating flow_params. Make sure the dictionary keys are as specified.
flow_params = dict(
    # name of the experiment
    exp_tag=EXP_NAME,
    # name of the flow environment the experiment is running on
    env_name=AccelEnv,
    # name of the network class the experiment uses
    network=FigureEightNetwork,
    # simulator that is used by the experiment
    simulator="traci",
    # simulation-related parameters
    sim=SumoParams(sim_step=0.1, render=False),
    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        # length of one rollout
        horizon=HORIZON,
        additional_params=ADDITIONAL_ENV_PARAMS,
    ),
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=NetParams(additional_params=ADDITIONAL_NET_PARAMS),
    # vehicles to be placed in the network at the start of a rollout
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=InitialConfig(spacing="uniform", perturbation=1),
)

# Call the utility function make_create_env to be able to
# register the Flow env for this experiment
gym_name, create_env = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [5]:
# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4
)  # generating a string version of flow_params

config = {
    **DEFAULT_CONFIG,
    "num_workers": N_WORKERS,  # number of parallel workers
    "train_batch_size": HORIZON * N_ROLLOUTS,  # batch size
    "model": {
        **DEFAULT_CONFIG["model"],
        "fcnet_hiddens": [16, 16],
    },  # size of hidden layers in network
    "use_gae": True,  # using generalized advantage estimation
    "gamma": 0.999,  # discount rate
    "lambda": 0.97,
    "sgd_minibatch_size": min(
        16 * 1024, HORIZON * N_ROLLOUTS
    ),  # stochastic gradient descent
    "kl_target": 0.02,  # target KL divergence
    "num_sgd_iter": 10,  # number of SGD iterations
    "horizon": HORIZON,  # rollout horizon
    "framework": "tf2",
    "eager_tracing": True,
    "env_config": {
        "flow_params": flow_json,
        "run": RL_ALG_NAME,
    },
}


2022-09-04 18:08:33,883	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.agents.ppo.ppo::DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.algorithms.ppo.ppo::PPOConfig(...)` instead. This will raise an error in the future!


In [6]:
trials = run_experiments(
    {
        EXP_NAME: {
            "run": RL_ALG_NAME,
            "env": gym_name,
            "config": config.copy(),
            "checkpoint_freq": 10,  # number of iterations between checkpoints
            "checkpoint_at_end": True,  # generate a checkpoint at the end
            "max_failures": 999,
            "stop": {  # stopping conditions
                "training_iteration": 150,  # number of iterations to stop after
            },
        },
    }
)


2022-09-04 18:08:35,909	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Trial name,status,loc,iter,total time (s),ts,reward,num_recreated_wor...,episode_reward_max,episode_reward_min
PPO_AccelEnv-v0_1fa80_00000,TERMINATED,127.0.0.1:84495,150,1376.71,750600,102.781,0,103.799,101.938


(PPO pid=84495) 2022-09-04 18:08:40,832	INFO algorithm.py:1860 -- Executing eagerly (framework='tf2'), with eager_tracing=True. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
(PPO pid=84495) 2022-09-04 18:08:40,832	WARNING ppo.py:350 -- `train_batch_size` (5000) cannot be achieved with your other settings (num_workers=9 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 556.
(PPO pid=84495) 2022-09-04 18:08:40,833	INFO algorithm.py:351 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=84504) Metal device set to: Apple M1 Max
(RolloutWorker pid=84504) 
(RolloutWorker pid=84504) systemMemory: 64.00 GB
(RolloutWorker pid=84504) maxCacheSize: 24.00 GB
(RolloutWorker pid=84504) 
(RolloutWorker pid=84503) Metal device set to: Apple M1 Max
(RolloutWorker pid=84503) 
(RolloutWorker pid=84503) systemMemory: 64.00 GB
(RolloutWorker pid=84503) maxCacheSize: 24.00 GB
(RolloutWorker pid=84503) 
(RolloutWorker pid=84501) Metal device set to: Apple M1 Max
(RolloutWorker pid=84501) 
(RolloutWorker pid=84501) systemMemory: 64.00 GB
(RolloutWorker pid=84501) maxCacheSize: 24.00 GB
(RolloutWorker pid=84501) 
(RolloutWorker pid=84498) Metal device set to: Apple M1 Max
(RolloutWorker pid=84498) 
(RolloutWorker pid=84498) systemMemory: 64.00 GB
(RolloutWorker pid=84498) maxCacheSize: 24.00 GB
(RolloutWorker pid=84498) 
(RolloutWorker pid=84505) Metal device set to: Apple M1 Max
(RolloutWorker pid=84505) 
(RolloutWorker pid=84505) systemMemory: 64.00 GB
(RolloutWorker p

(RolloutWorker pid=84503) Warning: Vehicle 'human_8' performs emergency braking on lane 'right_0' with decel=10.28, wished=4.50, severity=1.28, time=9.10.
(RolloutWorker pid=84505) Warning: Vehicle 'human_8' performs emergency braking on lane 'right_0' with decel=10.46, wished=4.50, severity=1.32, time=9.30.
(PPO pid=84495) 2022-09-04 18:08:56,594	WARNING deprecation.py:47 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


...

Result for PPO_AccelEnv-v0_1fa80_00000:
  agent_timesteps_total: 680544
  counters:
    num_agent_steps_sampled: 680544
    num_agent_steps_trained: 680544
    num_env_steps_sampled: 680544
    num_env_steps_trained: 680544
  custom_metrics: {}
  date: 2022-09-04_18-29-40
  done: false
  episode_len_mean: 5000.0
  episode_media: {}
  episode_reward_max: 103.79902259922984
  episode_reward_mean: 102.78063993278272
  episode_reward_min: 101.93793493358653
  episodes_this_iter: 0
  episodes_total: 135
  experiment_id: 0e814ab2ba9c41d8b7aa032a01a2cb23
  hostname: Maxs-MacBook-Pro.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.0
          cur_lr: 4.999999873689376e-05
          entropy: 1.3334676027297974
          entropy_coeff: 0.0
          kl: 1.8772328985505737e-05
          policy_loss: -0.04187304526567459
          total_loss: 5.258872985839844
          vf_explained_var: -0.0014101117849349976
          v

(RolloutWorker pid=84503) Warning: Vehicle 'human_8' performs emergency braking on lane 'right_0' with decel=10.27, wished=4.50, severity=1.28, time=8010.70.
(RolloutWorker pid=84505) Warning: Vehicle 'human_8' performs emergency braking on lane 'right_0' with decel=10.45, wished=4.50, severity=1.32, time=8010.90.


Result for PPO_AccelEnv-v0_1fa80_00000:
  agent_timesteps_total: 725580
  counters:
    num_agent_steps_sampled: 725580
    num_agent_steps_trained: 725580
    num_env_steps_sampled: 725580
    num_env_steps_trained: 725580
  custom_metrics: {}
  date: 2022-09-04_18-31-02
  done: false
  episode_len_mean: 5000.0
  episode_media: {}
  episode_reward_max: 103.79945888203886
  episode_reward_mean: 102.78080550188763
  episode_reward_min: 101.93793493358653
  episodes_this_iter: 0
  episodes_total: 144
  experiment_id: 0e814ab2ba9c41d8b7aa032a01a2cb23
  hostname: Maxs-MacBook-Pro.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.0
          cur_lr: 4.999999873689376e-05
          entropy: 1.303743600845337
          entropy_coeff: 0.0
          kl: 3.9257480239029974e-05
          policy_loss: -0.03199172765016556
          total_loss: 4.8065996170043945
          vf_explained_var: 0.0009902060264721513
          vf

(RolloutWorker pid=84504) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84504) Quitting (on error).
(RolloutWorker pid=84503) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84503) Quitting (on error).
(RolloutWorker pid=84506) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84506) Quitting (on error).
(RolloutWorker pid=84499) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84499) Quitting (on error).
(RolloutWorker pid=84505) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84505) Quitting (on error).
(RolloutWorker pid=84498) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84498) Quitting (on error).
(RolloutWorker pid=84501) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(RolloutWorker pid=84501) Quitting (on error).
(RolloutWorker pid=84502) Error: tcpip::Socket::recvAndCheck @ recv: peer shutdown
(Rollou

[PPO_AccelEnv-v0_1fa80_00000]

In [9]:
print(trials[0].logdir)
!python ../flow/visualize/visualizer_rllib.py ${trials[0].logdir} 10

NameError: name 'trials' is not defined